# Intro
We are Guy Halevi and Orr Jungerman, final-year students with great interest in data science. We chose this workshop to extend our knowledge and practice major data science principles.
It's worth noting we live close to each other in middle-north Tel-Aviv (Old North neighborhood), a district notoriously known to lack parking options.
When we were tasked with choosing a topic for this project, we wanted to tackle something important and beneficial for us. Thus, predicting parking in some way was one of the first ideas coming to mind.
Luckily, we have been harvesting parking data about few parking lots for quite some time.


# The Problem

Our experience with parking lots in Tel-Aviv led us to believe that there are common recurring patterns. For example, on weekdays most parking lots are emptying in the morning and filling up at the evening due to day jobs. On the contrary, on weekends the parking lots tend to empty much later since most of the residents are not working and driving back to their families.
To back our experience with real and objective data, we have been collecting for the past 2 years the vacancy status of popular parking lots, on a per-minute basis.
We believe we can use this data to accurately predict the status of each parking lot, helping us to plan ahead our schedule.
We expect to find interesting anomalies due to holidays or special events (such as lockdowns, rain, etc.). To deal with such anomalies, we will enrich the data with more information such as recurring holidays and more.



# Our Data
The data we collected comes in a CSV format, where each row contains the vacancy status of 6 different parking lots at specific time.
The parking lots are: Basel, Asuta, Sheraton, Dan, Dubnov and Habima.
There are 6 different vacancy statuses: Free - significant number of spots, Few - parking lot is almost full, Full - no spots at all, Active - no indication at that time, Unknown/NaN - we had trouble collecting the status.
We started collecting data since July 19, with minute resolution. We managed to collect the data by writing a simple script that parses the website of each parking lot, for example http://www.ahuzot.co.il/Parking/ParkingDetails/?ID=3.

The following map shows the locations of the 6 lots:

![title](lots.jpg)

# Exploratory Data Analysis

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
from ipywidgets import HTML
from more_itertools import first
from collections import Counter
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from tqdm.auto import tqdm
import tensorflow as tf

tqdm.pandas()
np.random.seed(0)
RANDOM_STATE=0

In [ ]:
pd.options.display.max_rows = 10

LOT_NAMES = ['Basel', 'Asuta', 'Sheraton', 'Dan', 'Dubnov', 'Habima']
DATA_CSV_FILENAME = 'june21_parks.csv'

Let's start by viewing the raw data.<br>
We will use pandas to load the CSV into a dataframe:

In [ ]:
raw_df = pd.read_csv(DATA_CSV_FILENAME)
raw_df

### Data Assesment

First, we will copy our raw data, so we can modify the copy without altering the original data.

Let's change column names to more standard and conventient names, and set the right types to each column in order to be able to use it later.

In [ ]:
full_df = raw_df.copy()
full_df = full_df.rename(columns={
    'Day': 'day',
    'Hour': 'hour',
    'Minute': 'minute',
    'Date': 'date'
})
full_df['datetime'] = full_df['date'] + " " + full_df["hour"].astype(str) + ":" + full_df["minute"].astype(str)
full_df['datetime'] = pd.to_datetime(full_df['datetime'], format='%d/%m/%Y %H:%M')
full_df['datetime'] = full_df['datetime'].dt.tz_localize('Asia/Jerusalem', 'infer')
full_df.pop('date')
full_df.head()

As you can see, `datetime` column is now recognized as a date object instead of a simple string.

Now, let's explore the "parking statuses", or classes in our terminology:

In [ ]:
full_df[LOT_NAMES].apply(pd.value_counts)

We can see that in addition to the 3 main statuses <`Free`, `Few`, `Full`>, there are 3 additional "unknown" statuses that are quite rare which we want to exclude from our dataset - <`NaN`, `Active`, `Unknown`>.

In addition to excluding the 3 "unknown" statuses, we need to decide how to handle the 3 "main" statuses.
The meaning of `Few` is that there are some parking spots available, but just a few. It means that in some cases, if we checked the status a few seconds earlier or later than we actually did, we would have been able to get different results. So, it makes sense to smooth the data. For example, if the status was `Full` for a while, then `Few` for a few minutes and then `Full` again, it makes more sense to translate this `Few` to `Full` than to `Free`, and the same for the other way around.

In order to do that, we will start by treating `Few` as "the middle between `Free` and `Full`", or in other words, we will translate `Free -> 1`, `Few -> 0.5`, `Full -> 0`, as `Free` has 1 (a lot) of free parking, `Few` has 0.5 (few) free parking and `Full` has 0 (no) free parking.<br>
Later we will normalize each `0.5` point to either `0` or `1`, depending on its surroundings.

In [ ]:
for lot in LOT_NAMES:
    full_df[lot] = full_df[lot].replace({
        'Unknown': np.nan,
        'Active': np.nan,
        'Free': 1,
        'Few': 0.5,
        'Full': 0,
    }, inplace=False)

print('Full dataset:')
display(full_df)
print('Classes:')
display(full_df[LOT_NAMES].apply(pd.value_counts))

In order to work with the data, it would be much more convenient if each row had only one class.<br>
In addition, we will remove rows with the class `NaN` so that our data is clean. We didn't do it before to avoid removing a complete row because of one missing value at one parking lot.<br>
We use `melt()` to do that:

In [ ]:
melted_df = (
    full_df
    .melt(
        id_vars=[
            'datetime',
            'day',
            'hour',
            'minute',
        ],
        var_name='lot',
        value_name='class'
    )
    .dropna()
)
melted_df

### Data resampling
Our dataset is incomplete - sometimes the scraper didn't run and sometimes there were problems with the API. However, we can try to rectify some of the lost data by resampling and applying a moving average. This approach will also help us to reduce noise and small anomalies.<br>
Using moving average, the discrete availability class becomes a continious variable of the parking availability likelihood at that point of time.

In [ ]:
# Upsample to 1m
resampled_df = melted_df.set_index('datetime').groupby('lot').resample('60s').first()

# Drop `lot` column - upsampled datapoints are set to na, including lot
resampled_df = resampled_df.drop(columns=['lot'])

# Ungroup, so all the samples get the proper lot and datetime values
resampled_df = resampled_df.reset_index()

# Group upsampled data by lot
avg_df = resampled_df.groupby(by=['lot'])

# Apply moving average on upsampled data
avg_df = avg_df.rolling('1h', on='datetime', min_periods=5, center=True, win_type='gaussian')['class'].mean(std=0.5)

# Drop data points that we could not decide their availability
avg_df = avg_df.dropna().reset_index()

#Rename class column to availability
avg_df = avg_df.rename(columns={"class": "availability"})

print(f"number of original samples: {len(melted_df)}")
print(f"number of resampled samples: {len(avg_df)}")
print(f"rectified: {len(avg_df) - len(melted_df)} samples")

avg_df

### Threshold and Downsample
After we applied a moving average resulting in the availability feature, we'd like to define a threshold for parking status classification.
Since `Few` status was translated as `0.5`, we believe it should act as the threshold nubmer too.<br>
Every hour that its average is lower than `0.5` is an hour that was `Full` most of the time and will be normalized to `0`, and every hour that is higher than `0.5` will be normalized to `1`.<br>
Regarding hours with an average of exactly `0.5`, we decided to treat them as `Free` (i.e. `0`), as it means that for a whole hour, most of the times there were "at least a few spots available".<br>
For computational reasons and simplicity of the data analysis, we decided to downsample the dataset to 1H.

In [ ]:
avg_df['class'] = (avg_df['availability'] >= 0.5).astype(int)
avg_df

In [ ]:
hourly_df = avg_df[avg_df['datetime'].dt.minute == 0].copy()
hourly_df['datehour'] = hourly_df['datetime'].dt.date.astype(str) + " - " + hourly_df['datetime'].dt.hour.astype(str).str.zfill(2)
hourly_df = hourly_df.set_index('datehour')
hourly_df['date'] = hourly_df['datetime'].dt.normalize()
hourly_df

### Data Visualization

Now comes the part where we want to visualize our data. The visualizations are supposes to give us a grasp on our data and validate our hypothesis. 

Let's start by visualizing the classes distribution among the different lots.<br>
We will use a color palette that gives us the right intuition - 1 is "good" (there are free parking spots!), therefore green, and 0 is "bad", therefore red.

In [ ]:
GREEN_RED_PALETTE = {1.0: 'green', 0.0: 'red'}

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

_ = sns.histplot(
    data=hourly_df,
    x='lot',
    hue='class',
    multiple='dodge',
    palette=GREEN_RED_PALETTE,
    shrink=0.7,
).set_title('Class Distribution Per Parking Lot')

As the histogram shows, some lots have relatively equally distributed classes (e.g. Basel), while some have almost only one class (e.g. Habima).

We can also see that `Free` is the most common class in all lots.<br>
It's already some interesting information for us, as we personally feel like Basel, the one that's closest to where we both live.
<br><br><br>
We'd like to mention how difficult it is to visualize the data in a way that makes sense and that is intuitive. We tried many methods but all showed too much information that was not helpful in any way.

For example, let's show classes over time, for one specific month - January 2020:

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

sns.lineplot(
    marker='o',
    x='date',
    y='class',
    hue='lot',
    data=hourly_df[(hourly_df['datetime'].dt.year == 2020) & (hourly_df['datetime'].dt.month == 1)],
)


for item in plt.xticks()[1]:
    item.set_rotation(90)

As you can see, even after we filter out most data and stay with only one month of data (out of ~2 years that we have!) is too much for visualizing without any special treatment.

Then we came up with the idea of weekly patterns. As mentioned before, we believe the parking vacancy status recurrs every week. Let's transform our data to validate this hypothesis.

### Weekly Patterns
From our own knowledge and experience, we know that available parking spots change over the week in a periodic way that is very similar between different weeks. For example, every morning in the week days there is quite a lot of parking, and every evening there isn't. In the weekends it changes; Friday morning is busy but Friday evening is always super free (as most young people visit their parents outside of Tel Aviv, probably).

It's a very interesting hypothesis to start with.<br>
Let's use FFT (Fast Fourier Transform) to prove that these patterns actually exist:
#### FFT

In [ ]:
fft_df = hourly_df.copy().reset_index().set_index('datetime')['class']
fft_df = fft_df.resample('1h').first()
# fft ignores random noise
fft_df = fft_df.apply(lambda l: l if not np.isnan(l) else np.random.random())

fft = tf.signal.rfft(fft_df)
freqs = np.arange(0, len(fft))
total_hours = len(fft_df)
hours_in_week = 7 * 24
steps = freqs / total_hours * hours_in_week

plt.step(steps, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 3000)
plt.xlim([0.2, 7 * 24]) # Limits to about 1/month - 1/hour
_ = plt.xticks([14, 7, 1], labels=['1 / 12-hours', '1 / day', '1 / week'])

The FFT diagram definitely shows that there is a significant recurring pattern every day (which means that the time of day is very correlated to the parking availability status) and a smaller spike, yet still visible and significant, every week (which means that the day of the week is correlated with the status).

We expected the weekly pattern to be more significant, but we assume it's not as significant as we expected because we expect all 5 weekdays to be quite the same, and only weekends to be different.
<br><br><br>
Let's group the data by weeks (every week is defined as 00:00 on Sunday until 23:00 on Saturday). To simplify the data, we drop a few rows that are before the first whole week we have data for:

In [ ]:
FIRST_DAY_OF_FIRST_WHOLE_WEEK = pd.to_datetime(date(year=2019, month=7, day=14)).tz_localize('Asia/Jerusalem')
hourly_df = hourly_df[hourly_df['date'] >= FIRST_DAY_OF_FIRST_WHOLE_WEEK].copy()

hourly_df['week_first_day'] = (
    ((hourly_df['date'] - FIRST_DAY_OF_FIRST_WHOLE_WEEK).dt.days / 7)
    .astype(int)
    .apply(lambda week_idx: FIRST_DAY_OF_FIRST_WHOLE_WEEK + pd.to_timedelta(timedelta(weeks=week_idx)))
)

hourly_df['day_hour'] = hourly_df['date'].dt.day_name() + '-' + hourly_df['datetime'].dt.hour.astype(str).str.zfill(2)
hourly_df

Now that we have week information for each row, we can start showing weekly trends.

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(13, 8))
fig.suptitle('Average Parking Per Hour In The Week', fontsize=22)
gs = fig.add_gridspec(3, 3)
axes = [fig.add_subplot(gs[0, :])]
for i in range(len(LOT_NAMES)):
    axes.append(fig.add_subplot(gs[1 + i // 3, i % 3]))

ax = axes[0]
sns.lineplot(
    ax=ax,
    x='day_hour',
    y='class',
    data=hourly_df
)
ax.set_title('All Lots Combined')

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i + 1]
    g = sns.lineplot(
        ax=ax,
        x='day_hour',
        y='class',
        data=hourly_df[hourly_df['lot'] == lot]
    )
    g.axhline(0.5, color='red')
    ax.set_title(lot)

for ax in axes:
    ax.set_ylim(bottom=0, top=1)
    for i, label in enumerate(ax.get_xticklabels()):
        if i % 8 == 0:
            label.set_rotation(90)
        else:
            label.set_visible(False)

Now the visualization is much better and it actually starts to make sense!<br>
We finally have our first informative visualization, that supports our hypothesis of weekly trends.

Note the width of the lines. The wider the line is, the more variation there is in the classes at that point. We can see that all lines are relatively narrow, especially when comparing to the earlier unsuccessful visualization.<br>
The meaning of the narrow lines is that for every hour in a week, its classes are quite similar between all weeks in our data.

Even though it's a pretty good start, the visualization does not prove that the data is completely predictable using `lot` and `day_hour` features only. It only shows that it has good correlation.<br><br>

Now let's take a look from another point of view.

Instead of showing the average parking per hour in the week, among all weeks, let's show the average parking per week over time.<br>
To do so, we need to have the average parking status per week over time.

We define the parking status of a week as the average of all hours within the week.<br>
It's important to note that our `class` will no longer be one of 2 discrete options - it will become a continuous number. We will only use this data for this visualization and then come back to the earlier per-hour format.

In [ ]:
weekly_df = (
    hourly_df
    .groupby(by=['week_first_day', 'lot'])
    .agg({'class': 'mean'})
    .reset_index()
)
weekly_df

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(13, 8))
fig.suptitle('Average Parking Per Week Over Time', fontsize=22)
gs = fig.add_gridspec(3, 3)
axes = [fig.add_subplot(gs[0, :])]
for i in range(len(LOT_NAMES)):
    axes.append(fig.add_subplot(gs[1 + i // 3, i % 3]))

ax = axes[0]
sns.lineplot(
    ax=ax,
    x='week_first_day',
    marker='o',
    y='class',
    data=weekly_df
)
ax.set_title('All Lots Combined')

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i + 1]
    sns.lineplot(
        ax=ax,
        x='week_first_day',
        marker='o',
        y='class',
        data=weekly_df[weekly_df['lot'] == lot]
    )
    ax.set_title(lot)

for ax in axes:
    ax.set_ylim(bottom=0, top=1)
    for i, label in enumerate(ax.get_xticklabels()):
        label.set_rotation(90)

As we can see, there have been some changes over time.<br>
There are several specific things to take from this view.

The first one is that there is a big gap without any data points somewhere between 2020-05 and 2020-07. Not only that, but the next week after the gap is extermely different than usual, mosly to the worse, but not only, and it differs a lot among the different lots as shown by the width of the top line.

The second is the variation around 2019-11, as shown by the width of the top line. Some of the parking lots had much more space than usual, and some much less.

The third is the two peak points around 2020-04 and 2020-10, in which all lots had unusual big amount of free parking spots.

We will try to use these insights later in this project, and will also try to explain them.<br><br>

Finally, we had an idea for a visualization that combines most data, and visualizes it in a way that makes much sense and is very intuitive.<br>
The idea is to show every week of a specific parking lot as a row of points, each has a color as before (green means `Free`, red means `Full`). To show trends over time, we put weeks above other ones, so that the lowest line is the earliest week:

In [ ]:
fig, axes = plt.subplots(len(LOT_NAMES), 1, figsize=(15, 8 * len(LOT_NAMES)), sharey=True)
fig.suptitle('Parking Per Hour In The Week Over Time', fontsize=22, y=0.895)
plt.subplots_adjust(hspace=0.4)

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i]
    plot = sns.scatterplot(
        ax=ax,
        palette=GREEN_RED_PALETTE,
        legend=False,
        x='day_hour',
        y='week_first_day',
        hue='class',
        s=10,
        data=hourly_df[hourly_df['lot'] == lot]
    )
    ax.set_title(lot)

for ax in axes:
    for i, label in enumerate(ax.get_xticklabels()):
        if i % 4 == 0:
            label.set_rotation(90)
        else:
            label.set_visible(False)

Perfect! We can see here most insights from both visualizations above.

The weekly trends are vertical trends (green / red "columns"), and changes over time are horizontal trends, like complete rows (weeks) that are completely green, that demonstrate complete weeks with only `Free` statuses.

# Classification Using Basic Data

Before we begin, we need to choose the features we want, and handle their types.<br>
Currently, the features we have look like that:

In [ ]:
clean_hourly_df = hourly_df.copy().reset_index()[['lot', 'datetime', 'class']]
clean_hourly_df

If we want the models to be able to use the inner information of `datetime`, we need to create columns describing the day and the time of the day.

For day, the simplest option is a one-hot representation, with a column for every day of the week (using `pd.get_dummies`).<br>
For hour, the simplest option is simply the numeric representation between 0-23.

In [ ]:
normalized_df = clean_hourly_df.copy()
normalized_df['day'] = normalized_df['datetime'].dt.day_name()
normalized_df['hour'] = normalized_df['datetime'].dt.hour
normalized_df = normalized_df.set_index(['lot', 'datetime'])
normalized_df = pd.get_dummies(normalized_df, prefix='', prefix_sep='')
normalized_df = normalized_df.reset_index()
normalized_df

When handling with a classification task, it is important to split wisely train and test data.

Usually, in non-time-based problems, the best practice is to split the data randomly.<br>
However, in our case, we want to simulate reality, in which we have prior data and we want to predict the future.

For this reason, we start by splitting the data in a way that `train` is all data before 31/12/2020, and `test` is all the data from 2021. <br>

In addition, we split our data by parking lot name, so we can train different models on each seperately. 

In [ ]:
TRAIN_TEST_SPLIT_DATE = pd.to_datetime(date(year=2021, month=1, day=1)).tz_localize('Asia/Jerusalem')

dataset_per_lot = {}
for lot_name, lot_df in normalized_df.groupby('lot'):
    train_data = lot_df[lot_df['datetime'] < TRAIN_TEST_SPLIT_DATE].drop(columns=['lot', 'datetime'])
    test_data = lot_df[lot_df['datetime'] >= TRAIN_TEST_SPLIT_DATE].drop(columns=['lot', 'datetime'])
    dataset_per_lot[lot_name] = (train_data, test_data)

train_values = sum([len(x[0]) for x in dataset_per_lot.values()])
test_values = sum([len(x[1]) for x in dataset_per_lot.values()])
print(f'Train data has {train_values} values')
print(f'Test data has {test_values} values')
print(f'Test data is {100 * test_values / train_values:.2f}% of all data')

### Initial LogisticRegression

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

def split_df_to_x_y(df): 
    df = df.copy()
    return df, df.pop('class')

scores = list()
for lot, (train, test) in dataset_per_lot.items():
    train_x, train_y = split_df_to_x_y(train)
    test_x, test_y = split_df_to_x_y(test)
    
    clf = LogisticRegression(random_state=RANDOM_STATE, max_iter=1e9)
    clf.fit(train_x, train_y)
    score = accuracy_score(y_true=test_y, y_pred=clf.predict(test_x))
    scores.append({'lot':lot, 'score':score})

scores_df = pd.DataFrame.from_dict(scores).set_index('lot')
scores_df.loc['Average'] = scores_df['score'].mean()
print(scores_df.to_string())

The accuracy results of this basic logistic regression are very diverse: the accuracy in Basel is as good as random (and worse than choosing the most common class) and the one in Dubnov is way higher than it's fair to expect.

When you think about it, there are a few reasons for that, which are quite clear.<br>
The first and main reason is that we chose a random point in time, and measured ourselves according to this data split only. This point in time is much too meaningfull - if we chose another point in time we would get completely different results.<br>
The second reason is that we used less that 2/3 of the data for training, and it might have not been enough. We should use more data for training and test ourselves on less.<br>
The third is related to the first and is specific to Dubnov. It seems that Dubnov lot was nearly only free during the whole year of 2021 so far, so a simple calssifier that always predicts "free" would have almost 100% accury. This is not True for all the time before 2021, though.

### Better Data Split
To avoid giving too much of a meaning to one chosen splitting point, we will use a technique similar to the classic k-folds cross validation.

We will leave some of the data aside as the `test` set, and use all other data for training and validating the results.<br>
The train-validation data will then be used in 10 different splits, each containing a 12-month period `train` set and a 1-month period `validation` set, of the month following the `train` year.

Each model will be trained and tested 10 different times (independently), and its score will be the average accuracy score of all 10 times.<br>
After comparing different models and different parameters, and choosing the best one according to its scores over the train-validation data, we will train the chosen model on all data before the `test` set starts, and evaluate its results on the `test` set.

In [ ]:
MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION = [
    (7, 2020), (8, 2020), (9, 2020), (10, 2020), (11, 2020), (12, 2020), (1, 2021), (2, 2021), (3, 2021), (4, 2021)
]
TEST_SPLIT_MONTH_AND_YEAR = (5, 2021)

def create_train_validation_test_datasets(df, print_results=False):
    train_and_validation_datasets_per_lot = []
    for month, year in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION:
        current_dataset_per_lot = {}
        for lot_name, lot_df in df.groupby('lot'):
            starting_train_date = pd.to_datetime(date(year=year - 1, month=month, day=1)).tz_localize('Asia/Jerusalem')
            starting_validation_date = pd.to_datetime(date(year=year, month=month, day=1)).tz_localize('Asia/Jerusalem')
            if month == 12:
                ending_validation_date = pd.to_datetime(date(year=year + 1, month=1, day=1)).tz_localize('Asia/Jerusalem')
            else:
                ending_validation_date = pd.to_datetime(date(year=year, month=month + 1, day=1)).tz_localize('Asia/Jerusalem')
            train_data = lot_df[(lot_df['datetime'] >= starting_train_date) & (lot_df['datetime'] < starting_validation_date)]
            validation_data = lot_df[(lot_df['datetime'] >= starting_validation_date) & (lot_df['datetime'] < ending_validation_date)]
            current_dataset_per_lot[lot_name] = (
                train_data.drop(columns=['lot', 'datetime']), validation_data.drop(columns=['lot', 'datetime'])
            )
        train_and_validation_datasets_per_lot.append(current_dataset_per_lot)

    test_dataset_per_lot = {}
    for lot_name, lot_df in df.groupby('lot'):
        month, year = TEST_SPLIT_MONTH_AND_YEAR
        starting_test_date = pd.to_datetime(date(year=year, month=month, day=1)).tz_localize('Asia/Jerusalem')
        ending_test_date = pd.to_datetime(date(year=year, month=month + 1, day=1)).tz_localize('Asia/Jerusalem')
        train_data = lot_df[lot_df['datetime'] < starting_test_date]
        test_data = lot_df[(lot_df['datetime'] >= starting_test_date) & (lot_df['datetime'] < ending_test_date)]
        test_dataset_per_lot[lot_name] = (
            train_data.drop(columns=['lot', 'datetime']), test_data.drop(columns=['lot', 'datetime'])
        )

    if print_results:
        print(f'Folds in train-validation data: {len(train_and_validation_datasets_per_lot)}')
        print(f'Shape of example train data for validation: {train_and_validation_datasets_per_lot[0]["Basel"][0].shape}')
        print(f'Shape of example validation data: {train_and_validation_datasets_per_lot[0]["Basel"][1].shape}')
        print(f'\nShape of example train data for test data: {test_dataset_per_lot["Basel"][0].shape}')
        print(f'Shape of example test data: {test_dataset_per_lot["Basel"][1].shape}')
    
    return train_and_validation_datasets_per_lot, test_dataset_per_lot


def _evaluate_model_once(model_training_func, model_evaluating_func, dataset_per_lot, use_test_as_val=False):
    scores = {}
    for lot, (train, test) in dataset_per_lot.items():
        train_x, train_y = split_df_to_x_y(train)
        test_x, test_y = split_df_to_x_y(test)
        if use_test_as_val:
            model = model_training_func(train_x, train_y, lot, val_x=test_x, val_y=test_y)
        else:
            model = model_training_func(train_x, train_y, lot)
        score = model_evaluating_func(model, test_x, test_y)
        scores[lot] = score
    return scores

def evaluate_model(model_training_func, model_evaluating_func, datasets_per_lot,
                   print_results=True, detailed=False, row_names=None, use_test_as_val=False):
    score_dicts = []
    if print_results:
        datasets_per_lot = tqdm(datasets_per_lot)
    for dataset_per_lot in datasets_per_lot:
        score_dicts.append(_evaluate_model_once(model_training_func, model_evaluating_func, dataset_per_lot, use_test_as_val=use_test_as_val))
    lot_to_scores = {lot: [score_dict[lot] for score_dict in score_dicts] for lot in LOT_NAMES}
    scores_df = pd.DataFrame.from_dict(lot_to_scores)
    if row_names:
        scores_df.index = row_names
    if print_results and detailed:
        scores_df['Average'] = scores_df.mean(axis=1)
        scores_df.loc['Average'] = scores_df.mean()
        plt.figure(figsize=(5,5))
        sns.heatmap(scores_df, annot=True)
        plt.show()
    scores_df = scores_df.mean().T
    scores_df.loc['Average'] = scores_df.mean()
    if print_results:
        print(scores_df.to_string())
    return scores_df.to_dict()

In [ ]:
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(normalized_df,
                                                                                                    print_results=True)

As we planned, a random fold has ~12 times more data in its `train` set than its `validation` one, and the final `test` set has similar amount of data to a typical `validation` set.

Now let's run the same LogisticRegression model and see its average validation score.

In [ ]:
results_df = pd.DataFrame(columns=['average score'])
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: (LogisticRegression(random_state=RANDOM_STATE, max_iter=1e9)
                                                       .fit(train_x, train_y)),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['initial logistic regression'] = score['Average']
results_df

Now it makes much more sense!<br>
For example, Basel's score is much higher than 50%, and Dubnov's is not 99.9% anymore.
<br><br><br>
Now that we have a good way to measure the models, let's try DecisionTree and RandomForest and compare them to the initial LogisticRegression.

### InitialDecisionTreeClassifier

In [ ]:
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: (DecisionTreeClassifier(max_depth=4, random_state=RANDOM_STATE)
                                                       .fit(train_x, train_y)),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['initial decision tree'] = score['Average']
results_df

#### Example DecisionTree Decision Making

In [ ]:
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(*split_df_to_x_y(train_and_validation_datasets_per_lot[0]['Basel'][0]))
plt.figure(figsize=(15,5))

_ = tree.plot_tree(
    clf, 
    feature_names=[col for col in train_and_validation_datasets_per_lot[0]['Basel'][0].columns if col != 'class'],
    proportion=True,
    impurity=True,
    filled=True,
    fontsize=10,
    class_names=['Full', 'Free']
)
plt.show()

This graph describes the decision process of the example Decision Tree model.<br>Each node contains classifying criteria such as hour or day of the week. The root of the graph is the starting point and then we follow its classification rules down to the nodes. We go left when the critiera are met, and right if not.<br>
For example, let's say we wan't to predict the status at Friday's evening (6pm). First node classifies by threshold hour<=17.5. Since our hour does not meet with this criterion, we take the right path. Then the criterion is Friday<=0.5 ("not Friday") so we go right again. Finally we reach the final leaf that tells us our predicted class is Free.<br>
Each node contains extra information that tells us about the certainty of each decision node. Gini score implies the probability of a class to belong to multiple classes. A gini score of 0 means that the decision node is 100% sure there is only one single class. Samples is the percentage of trained data that reached the corresponding decision node. Value describes how the trained data is distributed among the classes.

### Initial RandomForestClassifier

In [ ]:
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: (RandomForestClassifier(max_depth=4, n_estimators=10, random_state=RANDOM_STATE)
                                                       .fit(train_x, train_y)),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['initial random forest'] = score['Average']
results_df

Let's now take it another step forward, and compare more models with more parameter combinations to find the best one for each lot:

### chossing the best classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB


all_clf_confs = [
    (LogisticRegression, dict(max_iter=1e9, random_state=RANDOM_STATE)),
    (DecisionTreeClassifier, dict(max_depth=4, random_state=RANDOM_STATE)),
    (DecisionTreeClassifier, dict(max_depth=6, random_state=RANDOM_STATE)),
    (DecisionTreeClassifier, dict(max_depth=8, random_state=RANDOM_STATE)),
    (RandomForestClassifier, dict(max_depth=4, n_estimators=10, random_state=RANDOM_STATE)),
    (RandomForestClassifier, dict(max_depth=6, n_estimators=10, random_state=RANDOM_STATE)),
    (RandomForestClassifier, dict(max_depth=8, n_estimators=10, random_state=RANDOM_STATE)),
    (KNeighborsClassifier, dict(n_neighbors=3)),
    (KNeighborsClassifier, dict(n_neighbors=10)),
    (KNeighborsClassifier, dict(n_neighbors=100)),
    (AdaBoostClassifier, dict(random_state=RANDOM_STATE)),
    (GaussianNB, dict()),
]

def find_lot_to_best_clf(clf_confs):
    lot_to_max_score = {lot: -1 for lot in LOT_NAMES}
    lot_to_best_clf = {lot: None for lot in LOT_NAMES}
    for clf_class, clf_kwargs in tqdm(clf_confs, desc='Finding best classifier for each lot'):
        lot_to_score = evaluate_model(
            model_training_func=lambda train_x, train_y, lot: clf_class(**clf_kwargs).fit(train_x, train_y),
            model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
            datasets_per_lot=train_and_validation_datasets_per_lot,
            print_results=False
        )
        for lot in LOT_NAMES:
            if lot_to_score[lot] > lot_to_max_score[lot]:
                lot_to_max_score[lot] = lot_to_score[lot]
                lot_to_best_clf[lot] = (clf_class, clf_kwargs)
    return lot_to_best_clf

In [ ]:
lot_to_best_clf = find_lot_to_best_clf(all_clf_confs)

for lot, (cls, kwargs) in lot_to_best_clf.items():
    params = {k: v for k, v in kwargs.items() if k != 'random_state'}
    print(f'{lot}:{" " * (15 - len(lot))}{cls.__name__}{" " * (30 - len(cls.__name__))}(params: {params})')

As the results show, the best classifiers is not absolute and every lot has its best configuration.

In [ ]:
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: lot_to_best_clf[lot][0](**lot_to_best_clf[lot][1]).fit(train_x, train_y),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['initial best classifier'] = score['Average']
results_df

This score is amazing, comparing to the initial logistic regression we started with. By changing nothing but models and params, the accuracy score was improved by almost 8%!

### Advanced Time Normalization

The first improvement we want to try is to change the representation of the hour and the day from a numeric value of 0-23 and a one-hot representation, to a continuous representation in which the difference between the hours 23 and 1 is the same as the difference between 1 and 3, and the same with days of the week.<br>
To do so, we will create a sinus and a cosinus of the day and the week, and instead of saving the hour or the day, we will save a point on the wave that represents the same information.

In [ ]:
SECONDS_IN_DAY = 60 * 60 * 24
SECONDS_IN_WEEK = SECONDS_IN_DAY * 7

sin_normalized_df = clean_hourly_df.copy()
ts = sin_normalized_df['datetime'].astype('int64') // 10**9
sin_normalized_df['day sin'] = np.sin(ts * (2 * np.pi / SECONDS_IN_DAY))
sin_normalized_df['day cos'] = np.cos(ts * (2 * np.pi / SECONDS_IN_DAY))
sin_normalized_df['week sin'] = np.sin(ts * (2 * np.pi / SECONDS_IN_WEEK))
sin_normalized_df['week cos'] = np.cos(ts * (2 * np.pi / SECONDS_IN_WEEK))
display(sin_normalized_df)

train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)

In [ ]:
lot_to_best_clf = find_lot_to_best_clf(all_clf_confs)
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: lot_to_best_clf[lot][0](**lot_to_best_clf[lot][1]).fit(train_x, train_y),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['advanced time norm best classifier'] = score['Average']
results_df

The results are surprising - the accuracy with the original representation of the days and hours led to higher accuracy.
So, we will go back to using the original representation for now.

# Adding More Data

In order to enhance our results, we want to add as much relevant information as possible to the dataset.<br>
We will use the following data:
- Weather
- Day / night hours
- Holidays

All the data was taken from the internet using some good search and crawling methods that won't be detailed here, and saved to different CSV files.

We are going to use each of the files separately, find its impact, and in the end, merge everything together.

### Weather

In [ ]:
initial_weather_df = pd.read_csv('weather.csv')
initial_weather_df.head(3)

In [ ]:
initial_weather_df['Conditions'].value_counts()

The weather dataset includes a lot of information, most of it seems irrelevant or duplicated.
Our gut feeling was that rain would have a high impact on parking ratio, and maybe temperature as well, as people change habbits in different weather conditions, for example, go to the beach (and park near it) more as the weather is hotter.

To explore the data, let's first merge it with our main dataframe and look for correlations with `class` column:

In [ ]:
initial_weather_df['hour_datetime'] = (
    initial_weather_df['Date time']
    .apply(lambda d: datetime.strptime(d, '%m/%d/%Y %H:%M:%S'))
    .dt.tz_localize('Asia/Jerusalem', 'infer')
)
initial_weather_df['Conditions'] = initial_weather_df['Conditions'].map({
    'Partially cloudy': 'Partially cloudy',
    'Clear': 'Clear',
    'Rain, Partially cloudy': 'Rain',
    'Rain, Overcast': 'Rain',
    'Rain': 'Rain',
})
weather_research_df = normalized_df.copy()
weather_research_df['hour_datetime'] = weather_research_df['datetime'].copy().dt.floor('H', ambiguous=True)
weather_research_df = (
    weather_research_df
    .merge(initial_weather_df, on='hour_datetime', how='left')
    .drop(columns=['hour_datetime'])
)
weather_research_df = pd.get_dummies(weather_research_df, columns=['Conditions'])

In [ ]:
weather_corr_df = pd.DataFrame(weather_research_df.corr()['class'].sort_values(key=abs, ascending=False))
weather_corr_df = weather_corr_df.reset_index()
weather_corr_df = weather_corr_df.rename(columns={'index': 'column', 'class': 'corr with class'})
weather_corr_df['non none ratio'] = weather_corr_df['column'].apply(lambda col: (
    len(weather_research_df[~weather_research_df[col].isna()]) / len(weather_research_df)
))
with pd.option_context("display.max_rows", 50):
    display(weather_corr_df)

The results are surprising! `Heat Index` and `Cloud Cover` are more correlated to `class` than `Conditions_Rain`!
The next step is to actually use the data to enhance the classification. In order to do that, we will choose some of the features. We will use only features that have a decent correlation, a decent existance ratio, and make at least some sense to us (as oppose to `Wind Direction`, which simply does not).<br>
Therefore we will only take the features `Heat Index`, `Cloud Cover` and `Conditions`.
Temperature has such a low correlation that it will not be included!

In [ ]:
initial_weather_df[['Heat Index', 'Cloud Cover']].describe()

To get rid of `None`s, we will fill none values with the minimum value of each of the columns.

In [ ]:
weather_df = normalized_df.copy()
weather_df['hour_datetime'] = weather_df['datetime'].copy().dt.floor('H', ambiguous=True)
weather_df = (
    weather_df
    .merge(initial_weather_df[['hour_datetime', 'Heat Index', 'Cloud Cover', 'Conditions']], on='hour_datetime', how='left')
    .drop(columns=['hour_datetime'])
)
weather_df['Heat Index'] = weather_df['Heat Index'].fillna(weather_df['Heat Index'].min())
weather_df['Cloud Cover'] = weather_df['Cloud Cover'].fillna(weather_df['Cloud Cover'].min())
weather_df = pd.get_dummies(weather_df, columns=['Conditions'], prefix='', prefix_sep='')
display(weather_df)

train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(weather_df)

In [ ]:
lot_to_best_clf = find_lot_to_best_clf(all_clf_confs)
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: lot_to_best_clf[lot][0](**lot_to_best_clf[lot][1]).fit(train_x, train_y),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot
)
results_df.loc['weather best classifier'] = score['Average']
results_df

There we go! The accuracy is improved by ~0.1% thanks to the new data.

### Day / night hours

In [ ]:
initial_sun_hours_df = pd.read_csv('sun_hours.csv')
initial_sun_hours_df

The day / night dataset includes the sunrise and sunset exact minute for each day.
Our initial expectation was that it would have a high correlation with our `class`, because people change habbits according to the daylight hours, especially in the evening (i.e. we expect `sunset` to be more informative than `sunrise`).

To be able to use the data, let's normalize it, merge with our main dataframe and check its correlation with `class`:

In [ ]:
initial_sun_hours_df['sunrise'] = initial_sun_hours_df['sunrise'].str.split(':').apply(lambda tup: int(tup[0]) * 24 + int(tup[1]))
initial_sun_hours_df['sunset'] = initial_sun_hours_df['sunset'].str.split(':').apply(lambda tup: int(tup[0]) * 24 + int(tup[1]))
initial_sun_hours_df['sunrise'] = (initial_sun_hours_df['sunrise'] - initial_sun_hours_df['sunrise'].mean()) / (initial_sun_hours_df['sunrise'].max() - initial_sun_hours_df['sunrise'].min())
initial_sun_hours_df['sunset'] = (initial_sun_hours_df['sunset'] - initial_sun_hours_df['sunset'].mean()) / (initial_sun_hours_df['sunset'].max() - initial_sun_hours_df['sunset'].min())
initial_sun_hours_df['date'] = initial_sun_hours_df['date'].astype('datetime64[ns]')
initial_sun_hours_df

In [ ]:
sun_hours_df = normalized_df.copy()
sun_hours_df['date'] = sun_hours_df['datetime'].dt.date.astype('datetime64[ns]')
sun_hours_df = (
    sun_hours_df
    .merge(initial_sun_hours_df, on='date', how='left')
    .drop(columns=['date'])
)

print(sun_hours_df.corr('spearman')['class'].sort_values(key=abs, ascending=False).to_string())

Surprisingly, `sunrise` is the one that has a much higher correlation than sunset.
Let's see how it helps our classifiers:

In [ ]:
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sun_hours_df.drop(columns=['sunset']))
lot_to_best_clf = find_lot_to_best_clf(all_clf_confs)
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: lot_to_best_clf[lot][0](**lot_to_best_clf[lot][1]).fit(train_x, train_y),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot
)
results_df.loc['day night best classifier'] = score['Average']
results_df

It seems that this new information only confuses the classifiers.
Let's double check that adding sunset back doesn't help:

In [ ]:
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sun_hours_df)
lot_to_best_clf = find_lot_to_best_clf(all_clf_confs)
_ = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: lot_to_best_clf[lot][0](**lot_to_best_clf[lot][1]).fit(train_x, train_y),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot
)

Looks like we made the right decision leaving sunset out, but anyway, the new data does not help.
We will leave it out.

### Holidays

In [ ]:
initial_holiday_df = pd.read_csv('holidays.csv')
initial_holiday_df['date'] = initial_holiday_df.apply(lambda row: pd.Timestamp(date(year=row['year'], month=datetime.strptime(row['month'], '%b').month, day=row['day'])), axis=1)
initial_holiday_df = initial_holiday_df[['date', 'holiday_name', 'holiday_class']]
initial_holiday_df

What we have here is a list of all holidays in the relevant dates, including a large range of what can be called holidays:
- `no_vaction` - days that are special but don't make most people change their routines (e.g. Tisha B'Av, Tu Bishvat)
- `mainly_school_vacation` - days that are special and make some people change their routines, especially children (e.g. Hanukkah, Purim)
- `erev_hag` - days that their morning is a vacation for some people and the evening is for most people (e.g. Erev Rosh HaShana)
- `hol_hamoed` - days that are vacations for some people (e.g. Passover (Day 4))
- `all_vacation` - days that are vacations for almost everybody (e.g. Rosh HaShana)
- `elections` - days that are supposed to happen once in four years and happened four times in our two years of data

Let's merge the dataframe with our main one and see how the `class` changes during different holidays:

In [ ]:
holiday_df = normalized_df.copy()
holiday_df['date'] = holiday_df['datetime'].dt.date.astype('datetime64[ns]')
holiday_df = (
    holiday_df
    .merge(initial_holiday_df, on='date', how='left')
    .drop(columns=['date', 'holiday_name'])
)

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

_ = sns.histplot(
    data=holiday_df[~holiday_df['holiday_class'].isna()],
    x='holiday_class',
    hue='class',
    multiple='dodge',
    palette=GREEN_RED_PALETTE,
    shrink=0.7,
).set_title('Class Distribution Per Parking Lot')

Super interesting! It looks very indicative - `erev_hag` is mainly free, `all_vacation` and `hol_hamoed` are similar but a bit less, and `elections` are mainly full which is very unusual.

Let's validate our conclusions with correlation scores:

In [ ]:
holiday_df['holiday_class'] = holiday_df['holiday_class'].fillna('no_holiday')
holiday_df = holiday_df.set_index(['lot', 'datetime'])
holiday_df = pd.get_dummies(holiday_df)
holiday_df = holiday_df.reset_index()

print(holiday_df.corr()['class'].sort_values(key=abs, ascending=False).to_string())

Alright, the results show what we expected and we are ready to use it in our classification.

Let's get rid of the non-correlated ones and continue:

In [ ]:
holiday_df = holiday_df.drop(columns=['holiday_class_mainly_school_vacation', 'holiday_class_no_vacation'])

train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(holiday_df)

In [ ]:
lot_to_best_clf = find_lot_to_best_clf(all_clf_confs)
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: lot_to_best_clf[lot][0](**lot_to_best_clf[lot][1]).fit(train_x, train_y),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot
)
results_df.loc['holidays best classifier'] = score['Average']
results_df

As expected, the data did help the results. Not as much as we expected, but any accuracy increase is good news.

The next step is, obviously, merging the good from each dataset together and see the results:

### merging

In [ ]:
enriched_df = holiday_df.copy().merge(weather_df, on=[
    'lot', 'datetime', 'class', 'hour',
    'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'
])
display(enriched_df)

train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(enriched_df)

In [ ]:
lot_to_best_clf = find_lot_to_best_clf(all_clf_confs)
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: lot_to_best_clf[lot][0](**lot_to_best_clf[lot][1]).fit(train_x, train_y),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['merged data best classifier'] = score['Average']
results_df

The final result is certainly higher than the "initial best classifier"!

We are now ready to test our final classifier on the `test` data.

### Evaluating on TEST

In [ ]:
test_results_df = pd.DataFrame(columns=['average score'])
lot_to_best_clf = find_lot_to_best_clf(all_clf_confs)
score = evaluate_model(
    model_training_func=lambda train_x, train_y, lot: lot_to_best_clf[lot][0](**lot_to_best_clf[lot][1]).fit(train_x, train_y),
    model_evaluating_func=lambda model, test_x, test_y: accuracy_score(y_true=test_y, y_pred=model.predict(test_x)),
    datasets_per_lot=[test_dataset_per_lot]
)
test_results_df.loc['sklearn best classifiers'] = score['Average']

test_results_df

These results are amazing. Not only is the average score pretty high, but it is also very consistent with our validation results, which means we didn't overfit the training and validation data.

# Deep Learning
Let's utilize keras deep learning models to improve our prediction accuracy!

## Classification with Deep Learning
Before trying some fancy deep learning models, we should start with a simple DNN that attempts to predict parking availability status by our time features (sin/cos of day/week).

In [ ]:
from functools import partial

def _model_training_func(train_x, train_y, lot, model_creation_func, epochs=10, patience=2, 
                         loss_function=tf.losses.MeanSquaredError, metrics_function=tf.keras.metrics.BinaryAccuracy, 
                         optimizer_function=tf.optimizers.Adam):
    model = model_creation_func()
    train_x = np.array(train_x, dtype=np.float32)
    train_y = np.array(train_y, dtype=np.float32)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience, mode='min')
    model.compile(loss=loss_function(), 
                  optimizer=optimizer_function(),
                  metrics=[metrics_function()])
    model.fit(train_x, train_y, epochs=epochs, verbose=0, callbacks=[early_stopping])
    return model
    
def _model_evaluating_func(model, test_x, test_y):
    test_x = np.array(test_x, dtype=np.float32)
    test_y = np.array(test_y, dtype=np.float32)
    return model.evaluate(test_x, test_y, verbose=0)[1]

#### Linear DNN classifier

In [ ]:
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)

def _create_simple_clsf_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units=1)
    ])

score = evaluate_model(
    model_training_func=partial(_model_training_func, model_creation_func=_create_simple_clsf_model),
    model_evaluating_func=_model_evaluating_func,
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)

Great, we got decent prediction accuracy with the linear model. We might get better results if we add a hidden layer.
#### Dense DNN model

In [ ]:
def _create_dense_clsf_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])

score = evaluate_model(
    model_training_func=partial(_model_training_func, model_creation_func=_create_dense_clsf_model),
    model_evaluating_func=_model_evaluating_func,
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)

We got some really nice results! Let's see what happens when we add more hidden layers with dropout to prevent overfitting.

In [ ]:
def _create_dense2_clsf_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units=32),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.01),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)


score = evaluate_model(
    model_training_func=partial(_model_training_func, model_creation_func=_create_dense2_clsf_model),
    model_evaluating_func=_model_evaluating_func,
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['keras simple dense classifier'] = score['Average']

Adding more and more layer not necessarily improved accuracy. Some lots improved while other didn't. 
Let's try to add more features to the model, like we did before

In [ ]:
sin_enriched_df = (sin_normalized_df
                   .merge(enriched_df, on=["datetime", "lot", "class"])
                   .drop(columns=['hour', 'Friday', 'Monday', 'Saturday', 'Sunday','Thursday', 'Tuesday', 'Wednesday'])
                   .copy())

def _create_dense3_clsf_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units=4, activation='relu'),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])



train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_enriched_df)

score = evaluate_model(
    model_training_func=partial(_model_training_func, model_creation_func=_create_dense3_clsf_model, epochs=20, patience=5),
    model_evaluating_func=_model_evaluating_func,
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['keras enriched dense classifier'] = score['Average']

Overall, it seems like adding more features confused our model. Lots like Basel and Asuta are usually not affected by weather or vacations and we got less accurate predictions there. On the other hand, seasonal lots located near attractions such as Sheraton, Dan and Habima got a better prediction.<br>
Let's evaluate our best model on test data.

In [ ]:
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)


score = evaluate_model(
    model_training_func=partial(_model_training_func, model_creation_func=_create_dense2_clsf_model),
    model_evaluating_func=_model_evaluating_func,
    datasets_per_lot=[test_dataset_per_lot],
)
test_results_df.loc['keras simple dense classifier'] = score['Average']

## Time series forcasting with DNN
So far we only classified data based on their features without the context of their timely order.<br>
In the next section, we will create models that forecasts new timesteps based on prior information.<br>
The model's input will be `[input_timesteps, labels+features]` and output would be `[output_timesteps, labels]`.<br>
For example, a model that forecasts 3 hours into to future based on 6 hours with 4 labels (sin/cos of day/week) and 1 feature (class) would have input shape of `[6, 5]` and output shape of `[3, 1]`.<br>
To build such dataset, we will use `tf.keras.preprocessing.timeseries_dataset_from_array` that breaks the dataset into batches of consecutive timeseries with predefined lengths. Then we will split the input from the outputs.

In [ ]:
def _generate_dataset(df_x, df_y, input_timesteps, output_timesteps, 
                      y_labels_slice=slice(-1, None), add_y_features_to_x=False):
    # df_x and df_y split is irrelevant and we need to rebuild it
    df = df_x.copy()
    df['class'] = df_y
    
    # normalize dataframe types
    data_arr = np.array(df, dtype=np.float32)
    
    # create timeseries dataset
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data_arr,
        targets=None,
        sequence_stride=1,
        sequence_length=input_timesteps + output_timesteps, 
        shuffle=True,
        batch_size=32
    )
    
    def _split_x_y(batch):
        x = batch[:, :input_timesteps, :]
        y = batch[:, input_timesteps:, y_labels_slice]
        
        if add_y_features_to_x and input_timesteps == output_timesteps:
            y_vec = tf.unstack(batch[:, input_timesteps:, :], axis=2)
            del y_vec[y_labels_slice]
            y_features = tf.stack(y_vec, axis=2)
            x = tf.concat([x, y_features], axis=2)  
        
        # set shapes for clarity
        x.set_shape([None, input_timesteps, None])
        y.set_shape([None, output_timesteps, None])
        return x, y
    
    ds = ds.map(_split_x_y)
    
    return ds
    
def _ts_model_training_func(train_x, train_y, lot, model_creation_func, epochs=10, patience=2, input_timesteps=1, output_timesteps=1, fit_model=True, 
                            loss_function=tf.losses.MeanSquaredError, metrics_function=tf.keras.metrics.BinaryAccuracy, 
                            optimizer_function=tf.optimizers.Adam, y_labels_slice=slice(-1, None),
                            val_x=None, val_y=None, add_y_features_to_x=False):
    model = model_creation_func()
    dataset = _generate_dataset(train_x, train_y, input_timesteps, output_timesteps, y_labels_slice=y_labels_slice, add_y_features_to_x=add_y_features_to_x)
    callbacks = []
    dataset_val = None
    if val_x is not None:
        dataset_val = _generate_dataset(val_x, val_y, 
                                        input_timesteps, output_timesteps, 
                                        y_labels_slice=y_labels_slice, add_y_features_to_x=add_y_features_to_x)
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')]
    model.compile(loss=loss_function(), 
                  optimizer=optimizer_function(),
                  metrics=[metrics_function()])
    if fit_model:
        model.fit(dataset, epochs=epochs, verbose=0, validation_data=dataset_val)
    return model
    
def _ts_model_evaluating_func(model, test_x, test_y, 
                              input_timesteps=1, output_timesteps=1, 
                              y_labels_slice=slice(-1, None), add_y_features_to_x=False):
    dataset = _generate_dataset(test_x, test_y, 
                                input_timesteps, output_timesteps, 
                                y_labels_slice=y_labels_slice, add_y_features_to_x=add_y_features_to_x)
    return model.evaluate(dataset, verbose=0)[1]

### Single Step prediction
Let's start by predicting a data point just like before, but this the model will have access to previous hours.

#### Baseline Repeat
Before we create deep learning models, let's create a benchmark with a simple models that repeats over and over the input variables out

In [ ]:
class BaselineRepeat(tf.keras.Model):
    def __init__(self, output_timesteps=None):
        super().__init__()
        self._output_timesteps = output_timesteps
        
    def call(self, inputs):
#         return inputs[:, :, -1:]
        # inputs is a tensor with shape [batch, timesteps, features]
        input_timesteps = inputs.shape[1]
        output_timesteps = self._output_timesteps or input_timesteps
        
        multiplier = (output_timesteps // input_timesteps) + 1
        
        outputs = tf.concat([inputs]*multiplier, axis=1)[:, :output_timesteps, -1:]
        return outputs
    
def _create_baseline_repeat(output_timesteps=None):
    return BaselineRepeat(output_timesteps)
        

First, let's run the repeat baseline with 1 hour prediction. Model gets features and label of one hour and predicts the label of the next hour.

In [ ]:
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)


score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, model_creation_func=_create_baseline_repeat, input_timesteps=1 ,output_timesteps=1, fit_model=False),
    model_evaluating_func=partial(_ts_model_evaluating_func, output_timesteps=1),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['keras 1h prediction baseline'] = score['Average']

Great! we got really nice results here. Let's see if we can push this further using deep learning model.

In [ ]:
linear_ts_model = None
def create_linear_ts_model():
    global linear_ts_model
    linear_ts_model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=1)
    ])
    return linear_ts_model

score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_linear_ts_model, 
                                input_timesteps=1, output_timesteps=1,
                                add_y_features_to_x=True
                                ),
    model_evaluating_func=partial(_ts_model_evaluating_func, 
                                  input_timesteps=1, output_timesteps=1, 
                                  add_y_features_to_x=True),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=False
)

Thats's interesting, the simple dnn model could not beat the baseline model. Our model found a local optima that was not good enough. Let's dig into our model and see what it learned:

In [ ]:
linear_ts_model.weights

As expected, the most significant feature used for prediction was the last hour's label (class).<br>
Let's see what happens when we use different optimizer such as SGD.

In [ ]:
score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_linear_ts_model, 
                                input_timesteps=1 ,output_timesteps=1,
                                optimizer_function=tf.keras.optimizers.SGD,
                                add_y_features_to_x=True
                               ),
    model_evaluating_func=partial(_ts_model_evaluating_func, 
                                  input_timesteps=1, output_timesteps=1, 
                                  add_y_features_to_x=True),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)

In [ ]:
linear_ts_model.weights

This time, the model scored virtually the same as our baseline. When picking into its weights, we can see it gave the class feature more significance.<br>


Let's try again with another hidden layer and activation.

In [ ]:
dense_ts_model = None
def create_dense_ts_model():
    global dense_ts_model
    dense_ts_model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])
    return dense_ts_model

score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_dense_ts_model, 
                                input_timesteps=1, output_timesteps=1,
                                add_y_features_to_x=True
                                ),
    model_evaluating_func=partial(_ts_model_evaluating_func, 
                                  input_timesteps=1, output_timesteps=1, 
                                  add_y_features_to_x=True),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)

Mostly the same results as baseline.
### Multistep forecast
Next, we will attempt to predict multiple points instead of single prediction. 
#### Baseline
Let's see how well our baseline scores when it needs to predict a whole day based on its previous one.

In [ ]:
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)


score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=_create_baseline_repeat, 
                                input_timesteps=24 ,output_timesteps=24, fit_model=False),
    model_evaluating_func=partial(_ts_model_evaluating_func, 
                                  input_timesteps=24, output_timesteps=24),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION]
)
results_df.loc['keras 24h prediction baseline'] = score['Average']

Baseline model reached 82% accuracy! Like before, let's try to improve accuracy using deep learning models.
#### Simple DNN

In [ ]:
score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_dense_ts_model, 
                                input_timesteps=24, output_timesteps=24,
                                optimizer_function=tf.keras.optimizers.SGD, epochs=20,
                                add_y_features_to_x=True
                               ),
    model_evaluating_func=partial(_ts_model_evaluating_func, 
                                  input_timesteps=24, output_timesteps=24, 
                                  add_y_features_to_x=True),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)
results_df.loc['keras 24h prediction dense'] = score['Average']

This is great! Even though our model is still pretty basic, we scored with pretty decent accuracy!
#### CNN Model
Next, we will try to improve accuracy by granting our model access to multiple timesteps features each time. <br>
The input of our model is a 3d tensor with shape of`batch, timestep, features`. Dense layers operate only on the 3rd axis, so different timesteps are tuned separately.<br>
We can use `Flatten` or `Conv1D` layers to operate on multiple timesteps simultaneously.

In [ ]:
def create_ts_conv_model():
    return tf.keras.Sequential([
        tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(24)),
        tf.keras.layers.Dense(24, kernel_initializer=tf.initializers.zeros()),
        tf.keras.layers.Reshape([24, 1]),
    ])

train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)

score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_ts_conv_model, 
                                input_timesteps=24, output_timesteps=24,
                                optimizer_function=tf.keras.optimizers.SGD, epochs=20,
                                add_y_features_to_x=True
                               ),
    model_evaluating_func=partial(_ts_model_evaluating_func, 
                                  input_timesteps=24, output_timesteps=24, 
                                  add_y_features_to_x=True),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)
results_df.loc['keras 24h prediction cnn'] = score['Average']

Adding CNN layer did not improve our accuracy at all. Let's try another approach and use LSTM layer that learns "trends" with fewer trainable variables.

In [ ]:
def create_ts_lstm_model():
    return tf.keras.Sequential([
        tf.keras.layers.LSTM(32, return_sequences=False),
        tf.keras.layers.Dense(24, kernel_initializer=tf.initializers.zeros()),
        tf.keras.layers.Reshape([24, 1]),
    ])

train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)

score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_ts_lstm_model, 
                                input_timesteps=24, output_timesteps=24,
                                optimizer_function=tf.keras.optimizers.SGD, epochs=20,
                                add_y_features_to_x=True
                               ),
    model_evaluating_func=partial(_ts_model_evaluating_func, 
                                  input_timesteps=24, output_timesteps=24, 
                                  add_y_features_to_x=True),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)

Still not improving. The LSTM layer could hide the y features (sin/cos of day/week) we injected into our dataset. To overcome this issue, we can come up with non-sequential model that feeds the following Dense layer with the original y features input.<br>
To do so, we will create a custom class that derives from keras Model, and write our logic.

In [ ]:
class LstmWithYFeatures(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.lstm = tf.keras.layers.LSTM(32, return_sequences=False)
        self.dense = tf.keras.layers.Dense(32, activation='relu')
        self.dense_output = tf.keras.layers.Dense(1)
    
    def call(self, inputs, training=None):
        # extract only x-features
        x = inputs[:, :, :5]
        y_features = inputs[:, :, 5:]
        
        # (batch, steps, x_features) => (batch, lstm_units)
        lstm_units = self.lstm(x)
        
        # (batch, lstm_units) => (batch, 1, lstm_units)
        lstm_units = lstm_units[:, tf.newaxis, :]
        
        # (batch, 1, lstm_units) => (batch, steps, lstm_units)
        lstm_units = tf.concat([lstm_units]*24, axis=1)
        
        # (batch, steps, lstm_units) => (batch, steps, lstm_unit+y_features)
        lstm_with_y_features = tf.concat([lstm_units, y_features], axis=2)
        
        # (batch, steps, lstm_unit+y_features) => (batch, steps, dense_units)
        predictions = self.dense(lstm_with_y_features)
        
        # (batch, steps, dense_units) => (batch, steps, y_label)
        predictions = self.dense_output(predictions)
        
        return predictions

In [ ]:
def create_lstm2_model():
    return LstmWithYFeatures()

train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(sin_normalized_df)

score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_lstm2_model, 
                                input_timesteps=24, output_timesteps=24,
                                optimizer_function=tf.keras.optimizers.SGD, epochs=20,
                                add_y_features_to_x=True
                               ),
    model_evaluating_func=partial(_ts_model_evaluating_func, 
                                  input_timesteps=24, output_timesteps=24, 
                                  add_y_features_to_x=True),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)
results_df.loc['keras 24h prediction lstm'] = score['Average']

We managed to improve our 24h prediction accuracy even more! Lets see how we're doing so far:

In [ ]:
with pd.option_context("display.max_rows", 50):
    display(results_df.sort_values(by='average score'))

#### Autoregressive Models
In the previous section, we used single-shot models that predicted multiple steps simultaneously.<br>
Another approach we can try, is to create self-feeding model that predicts multiple steps, but one step at a time.<br>
For example, we can write a simple AR model based on simple dense layer with the following logic:
- gets 24h input
- predicts 25th hour
- predicts 26th hour based on 2-24h (from input) and 25h (from last prediction)
- predicts 27th hour based on 3-24h (from input) and 25-26h (from last predictions)
- and so forth... until we predict the 48th hour based on 24h (from input) and 25-47h (from last predictions)
- returns hours 25-48<br>
This model should be simpler to train because at each iteration the models only needs to learn single step.

In [ ]:
class DenseAR(tf.keras.Model):
    def __init__(self, units, in_steps, out_steps):
        super().__init__()
        self.in_steps = in_steps
        self.out_steps = out_steps
        self.units = units
        self.conv = tf.keras.layers.Conv1D(units, activation='relu', kernel_size=(in_steps))
        self.dense = tf.keras.layers.Dense(5)
        self._y_features = None
    
    def _extract_time_features(self, data):
        # this time we inject the relevant y features directly from the y dataset
        # the last models didn't do it and expected the inputs to be already injected.
        _, y, _ = tf.keras.utils.unpack_x_y_sample_weight(data)
        self._y_features = y[:, :, :-1]
        return data
    
    def train_step(self, data):
        self._extract_time_features(data)
        return super().train_step(data)
        
    def predict_step(self, data):
        self._extract_time_features(data)
        return super().predict_step(data)
    
    def test_step(self, data):
        self._extract_time_features(data)
        return super().test_step(data)
    
    def call(self, inputs, training=None, y_features=None):
        if y_features is not None:
            self._y_features = y_features[:, :, :-1]
        
        predictions = inputs
        # Run the rest of the prediction steps
        for n in range(self.out_steps):
            # (batch, steps, features) => (batch, conv_units)
            prediction = self.conv(predictions)
            
            # (batch, conv_units) => (batch, y_predicted_features + y_predicted_labels)
            prediction = self.dense(prediction)
            
            # (batch, y_predicted_features + y_predicted_labels) => (batch, y_features + y_predicted_labels)
            prediction = tf.concat([self._y_features[:,n,:], prediction[:, 0, -1:]], axis=1)

            # shift the inputs tensor right with last prediction
            predictions = tf.concat(
                (predictions[:, 1:, :], prediction[:, tf.newaxis, :]),
                axis=1
            )
            
        return predictions

Since our model returns features and labels (instead of just the single label - class), we need to customize our metrics and loss functions to ignore irrelevant labels.

In [ ]:
class LabelMSE(tf.losses.MeanAbsoluteError):
    def call(self, y_true, y_pred):
        y_true = y_true[:, :, -1]
        y_pred = y_pred[:, :, -1]
        return super().call(y_true, y_pred)
    
class LabelBinaryAccuracy(tf.keras.metrics.Accuracy):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = y_true[:, :, -1]
        y_pred = y_pred[:, :, -1]
        y_true = tf.greater_equal(y_true, 0.5)
        y_pred = tf.greater_equal(y_pred, 0.5)
        return super().update_state(y_true, y_pred, sample_weight)

In [ ]:
def create_ar_model():
    return DenseAR(128, 24, 24)

df = sin_normalized_df.copy()
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(df)


score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_ar_model, 
                                input_timesteps=24 ,output_timesteps=24, epochs=20,
                                loss_function=LabelMSE, metrics_function=LabelBinaryAccuracy, 
                                y_labels_slice=slice(None)),
    model_evaluating_func=partial(_ts_model_evaluating_func, input_timesteps=24, output_timesteps=24, y_labels_slice=slice(None)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)
results_df.loc['keras 24h prediction AR dense'] = score['Average']

Amazing! we pushed further and we're almost reaching 88%. This score seems promising, so we will evaluate it on test data<br>

In [ ]:
df = sin_normalized_df.copy()
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(df)


score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_ar_model, 
                                input_timesteps=24 ,output_timesteps=24, epochs=20,
                                loss_function=LabelMSE, metrics_function=LabelBinaryAccuracy,
                                y_labels_slice=slice(None)),
    model_evaluating_func=partial(_ts_model_evaluating_func, input_timesteps=24, output_timesteps=24, y_labels_slice=slice(None)),
    datasets_per_lot=[test_dataset_per_lot]
)
test_results_df.loc['keras 24h prediction AR dense'] = score['Average']

Next, we will try to run an LSTM model with the same autoregressive approach. We will create warm up step to prepare our lstm state, and then continue with feeding this model its own predictions.

In [ ]:
class LstmAR(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense2 = tf.keras.layers.Dense(32, activation='relu')
        self.dense = tf.keras.layers.Dense(5)
        self._y_features = None
    
    def _extract_time_features(self, data):
        _, y, _ = tf.keras.utils.unpack_x_y_sample_weight(data)
        self._y_features = y[:, :, :-1]
        return data
    
    def train_step(self, data):
        self._extract_time_features(data)
        return super().train_step(data)
        
    def predict_step(self, data):
        self._extract_time_features(data)
        return super().predict_step(data)
    
    def test_step(self, data):
        self._extract_time_features(data)
        return super().test_step(data)
    
    def warmup(self, inputs):
        # (batch, steps, features) => (batch, lstm_units), (batch, states)
        x, *state = self.lstm_rnn(inputs)
        
        
        
        prediction = self.dense2(x)
        prediction = self.dense(prediction)
        return prediction, state
    
    def call(self, inputs, training=None, y_features=None):
        if y_features is not None:
            self._y_features = y_features[:, :, :-1]
    
        predictions = []
        prediction, state = self.warmup(inputs)
        prediction = prediction[:,-1:]

        prediction = tf.concat([self._y_features[:,0,:], prediction], axis=1)
        # Insert the first prediction
        predictions.append(prediction)

        # Run the rest of the prediction steps
        for n in range(1, self.out_steps):
            
        # Use the last prediction as input.
            x = prediction
            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state,
                                      training=training)
            
            prediction = self.dense2(x)
            prediction = self.dense(prediction)
            prediction = prediction[:, -1:]

            prediction = tf.concat([self._y_features[:,n,:], prediction], axis=1)
            # Add the prediction to the output
            predictions.append(prediction)

        # predictions.shape => (time, batch, features)
        predictions = tf.stack(predictions)
        
                
        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])
        return predictions

In [ ]:
def create_lstm_ar_model():
    return LstmAR(32, 24)

df = sin_normalized_df.copy()
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(df)


score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_lstm_ar_model, 
                                input_timesteps=24 ,output_timesteps=24, epochs=20,
                                loss_function=LabelMSE, metrics_function=LabelBinaryAccuracy,
                                y_labels_slice=slice(None)),
    model_evaluating_func=partial(_ts_model_evaluating_func, input_timesteps=24, output_timesteps=24, y_labels_slice=slice(None)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)

In [ ]:
df = sin_normalized_df.copy()
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(df)


score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_lstm_ar_model, 
                                input_timesteps=24 ,output_timesteps=24, epochs=20,
                                loss_function=LabelMSE, metrics_function=LabelBinaryAccuracy,
                                y_labels_slice=slice(None)),
    model_evaluating_func=partial(_ts_model_evaluating_func, input_timesteps=24, output_timesteps=24, y_labels_slice=slice(None)),
    datasets_per_lot=[test_dataset_per_lot]
)

As we saw before, LSTM could potentialy hide important features such as y-features (day/week sin/cos), we can feed our dense layer directly with it.

In [ ]:
class LstmAR2(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense2 = tf.keras.layers.Dense(32, activation='relu')
        self.dense = tf.keras.layers.Dense(5)
        self._y_features = None
    
    def _extract_time_features(self, data):
        _, y, _ = tf.keras.utils.unpack_x_y_sample_weight(data)
        self._y_features = y[:, :, :-1]
        return data
    
    def train_step(self, data):
        self._extract_time_features(data)
        return super().train_step(data)
        
    def predict_step(self, data):
        self._extract_time_features(data)
        return super().predict_step(data)
    
    def test_step(self, data):
        self._extract_time_features(data)
        return super().test_step(data)
    
    def warmup(self, inputs):
        # (batch, steps, features) => (batch, lstm_units), (batch, states)
        x, *state = self.lstm_rnn(inputs)
        # (batch, features) => (batch, y_features+features)
        x = tf.concat([self._y_features[:,0,:], x], axis=1)
        prediction = self.dense2(x)
        prediction = self.dense(prediction)
        return prediction, state
    
    def call(self, inputs, training=None, y_features=None):
        if y_features is not None:
            self._y_features = y_features[:, :, :-1]
        
        # Use a TensorArray to capture dynamically unrolled outputs.
        predictions = []
        # Initialize the lstm state
        prediction, state = self.warmup(inputs)
        prediction = prediction[:,-1:]

        prediction = tf.concat([self._y_features[:,0,:], prediction], axis=1)
        # Insert the first prediction
        predictions.append(prediction)

        # Run the rest of the prediction steps
        for n in range(1, self.out_steps):
            
        # Use the last prediction as input.
            x = prediction
            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state,
                                      training=training)
            
            # Convert the lstm output to a prediction.
#             prediction = self.dense2(x)
#             prediction = self.dense3(x)
            x = tf.concat([self._y_features[:,n,:], x], axis=1)
            prediction = self.dense2(x)
            prediction = self.dense(prediction)
            prediction = prediction[:, -1:]

            prediction = tf.concat([self._y_features[:,n,:], prediction], axis=1)
            # Add the prediction to the output
            predictions.append(prediction)

        # predictions.shape => (time, batch, features)
        predictions = tf.stack(predictions)
        
                
        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])
        return predictions

In [ ]:
def create_lstm2_ar_model():
    return LstmAR2(32, 24)

df = sin_normalized_df.copy()
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(df)


score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_lstm2_ar_model, 
                                input_timesteps=24 ,output_timesteps=24, epochs=20,
                                loss_function=LabelMSE, metrics_function=LabelBinaryAccuracy,
                                y_labels_slice=slice(None)),
    model_evaluating_func=partial(_ts_model_evaluating_func, input_timesteps=24, output_timesteps=24, y_labels_slice=slice(None)),
    datasets_per_lot=train_and_validation_datasets_per_lot,
    detailed=True,
    row_names=[f'{m}/{y}' for m, y in MONTHS_AND_YEARS_FOR_TRAIN_AND_VALIDATION],
    use_test_as_val=True
)
results_df.loc['keras 24h prediction AR LSTM'] = score['Average']

We improved a little bit, still behind the CNN-Dense AR model. Let's evaluate this model on test.

In [ ]:
df = sin_normalized_df.copy()
train_and_validation_datasets_per_lot, test_dataset_per_lot = create_train_validation_test_datasets(df)


score = evaluate_model(
    model_training_func=partial(_ts_model_training_func, 
                                model_creation_func=create_lstm2_ar_model, 
                                input_timesteps=24 ,output_timesteps=24, epochs=20,
                                loss_function=LabelMSE, metrics_function=LabelBinaryAccuracy,
                                y_labels_slice=slice(None)),
    model_evaluating_func=partial(_ts_model_evaluating_func, input_timesteps=24, output_timesteps=24, y_labels_slice=slice(None)),
    datasets_per_lot=[test_dataset_per_lot]
)
test_results_df.loc['keras 24h prediction AR LSTM'] = score['Average']

# Classifier Results
We tackled our data with lots of classification approach. Let's summarize the results!
### Validation Results

In [ ]:
with pd.option_context("display.max_rows", 50):
    display(results_df)

### Test Results

In [ ]:
with pd.option_context("display.max_rows", 50):
    display(test_results_df)

## Accuracy and Anomalies
We can compare our predictions to the true values to find special patterns and anomalies which could help us in the future improve.<br>
First, let's take our sklearn best classifier we found before and train them with all our data.

In [ ]:
_, test_dataset_per_lot = create_train_validation_test_datasets(enriched_df)

classic_models = {lot: model[0](**model[1]) for lot, model in lot_to_best_clf.items()}
for lot, lot_df in tqdm(test_dataset_per_lot.items()):
    x, y = split_df_to_x_y(lot_df[0])
    classic_models[lot].fit(x, y)

Then, we can create special dataframe with success column

In [ ]:
predict_df = dict()
for lot_name, lot_df in enriched_df.groupby('lot'):
    df = lot_df.copy()
    x, y = split_df_to_x_y(lot_df.drop(columns=['lot', 'datetime']))
    y_predicts = classic_models[lot_name].predict(x)
    df['prediction'] = y_predicts
    df.loc[df['prediction'] == df['class'], 'success'] = 1
    df.loc[df['prediction'] != df['class'], 'success'] = 0
    df['date'] = df['datetime'].dt.normalize()
    df['week_first_day'] = (
        ((df['date'] - FIRST_DAY_OF_FIRST_WHOLE_WEEK).dt.days / 7)
        .astype(int)
        .apply(lambda week_idx: FIRST_DAY_OF_FIRST_WHOLE_WEEK + pd.to_timedelta(timedelta(weeks=week_idx)))
    )

    df['day_hour'] = df['date'].dt.day_name() + '-' + df['datetime'].dt.hour.astype(str).str.zfill(2)

    
    predict_df[lot_name] = df
    
predict_with_lot_name = predict_df.copy()
for lot in LOT_NAMES:
    predict_with_lot_name[lot]['lot'] = lot
    predict_with_lot_name[lot]

predict_with_lot_name = pd.concat(predict_with_lot_name.values())
predict_with_lot_name

## Visualize Accuracy
Just like we did at the beginning, we can use our data exploration visualizations to learn about our accuracy.

In [ ]:
fig, axes = plt.subplots(len(LOT_NAMES), 1, figsize=(15, 8 * len(LOT_NAMES)), sharey=True)
fig.suptitle('Model Accuracy Per Hour', fontsize=22, y=0.895)
plt.subplots_adjust(hspace=0.4)

for i, lot in enumerate(LOT_NAMES):
    df = predict_df[lot]
    ax = axes[i]
    plot = sns.scatterplot(
        ax=ax,
        palette=GREEN_RED_PALETTE,
        legend=False,
        x='day_hour',
        y='week_first_day',
        hue='success',
        s=10,
        data=df
    )
    ax.set_title(lot)

for ax in axes:
    for i, label in enumerate(ax.get_xticklabels()):
        if i % 4 == 0:
            label.set_rotation(90)
        else:
            label.set_visible(False)

On most of the parking lots, we can spot some patterns of missed predictions. It seems as the pattern is repeated on certain hours and days.<br>
Let's continue with our exploration and look for weekly patterns.

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(13, 8))
fig.suptitle('Average Accuracy Per Hour In The Week', fontsize=22)
gs = fig.add_gridspec(3, 3)
axes = [fig.add_subplot(gs[0, :])]
for i in range(len(LOT_NAMES)):
    axes.append(fig.add_subplot(gs[1 + i // 3, i % 3]))

ax = axes[0]
sns.lineplot(
    ax=ax,
    x='day_hour',
    y='success',
    data=predict_with_lot_name
)
ax.set_title('All Lots Combined')

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i + 1]
    g = sns.lineplot(
        ax=ax,
        x='day_hour',
        y='success',
        data=predict_with_lot_name[predict_with_lot_name['lot'] == lot]
    )
    ax.set_title(lot)

for ax in axes:
    ax.set_ylim(bottom=0.5, top=1)
    for i, label in enumerate(ax.get_xticklabels()):
        if i % 8 == 0:
            label.set_rotation(90)
        else:
            label.set_visible(False)

Intersting. We definitely see some hourly patterns here! Let's zoom in and group by the hour

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(13, 8))
fig.suptitle('Average Accuracy Per Hour', fontsize=22)
gs = fig.add_gridspec(3, 3)
axes = [fig.add_subplot(gs[0, :])]
for i in range(len(LOT_NAMES)):
    axes.append(fig.add_subplot(gs[1 + i // 3, i % 3]))

ax = axes[0]
sns.lineplot(
    ax=ax,
    x='hour',
    y='success',
    data=predict_with_lot_name
)
ax.set_title('All Lots Combined')

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i + 1]
    g = sns.lineplot(
        ax=ax,
        x='hour',
        y='success',
        data=predict_with_lot_name[predict_with_lot_name['lot'] == lot]
    )
    ax.set_title(lot)

for ax in axes:
    ax.set_ylim(bottom=0.5, top=1)
    for i, label in enumerate(ax.get_xticklabels()):
        label.set_rotation(90)

Now we got it! We can clearly see the unique patterns each lot has!<br>Let's see if we can spot any anomalies over time

In [ ]:
weekly_predict_df = (
    predict_with_lot_name
    .groupby(by=['week_first_day', 'lot'])
    .agg({'success': 'mean'})
    .reset_index()
)

fig = plt.figure(constrained_layout=True, figsize=(13, 8))
fig.suptitle('Average Accuracy Per Week Over Time', fontsize=22)
gs = fig.add_gridspec(3, 3)
axes = [fig.add_subplot(gs[0, :])]
for i in range(len(LOT_NAMES)):
    axes.append(fig.add_subplot(gs[1 + i // 3, i % 3]))

ax = axes[0]
sns.lineplot(
    ax=ax,
    x='week_first_day',
    marker='o',
    y='success',
    data=weekly_predict_df
)
ax.set_title('All Lots Combined')

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i + 1]
    sns.lineplot(
        ax=ax,
        x='week_first_day',
        marker='o',
        y='success',
        data=weekly_predict_df[weekly_predict_df['lot'] == lot]
    )
    ax.set_title(lot)

for ax in axes:
    ax.set_ylim(bottom=0, top=1)
    for i, label in enumerate(ax.get_xticklabels()):
        label.set_rotation(90)

In [ ]:
weekly_predict_df[weekly_predict_df['lot'] == 'Basel'].sort_values(by=['success']).head(10)

We can find some interesting anomalies on SOME of the data. For example the week of 21/03/20's was the beginning of the first lockdown

### Using our Deep learning models
We can sift out some minor anomalies if we check our DNN models with sklearn's models.<br>
First, we need to run and evaluate our model on all our data.

In [ ]:
def create_ar_model():
    return DenseAR(128, 24, 24)
def _split_x_y(batch):
    x = batch[:, :24, :]
    y = batch[:, 24:, :] 
    x.set_shape([None, 24, None])
    y.set_shape([None, 24, None])
    return x, y
    
models = {}
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2, mode='min')
                                                  
for lot_name, lot_df in tqdm(sin_normalized_df.groupby('lot')):
    lot_df = lot_df[['day sin', 'day cos', 'week sin', 'week cos', 'class']]
    arr = np.array(lot_df, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        arr,
        targets=None,
        sequence_stride=1,
        sequence_length=48, 
        shuffle=True,
        batch_size=32
    )
    ds = ds.map(_split_x_y)
    model = create_ar_model()
    model.compile(loss=LabelMSE(), 
                  optimizer=tf.optimizers.Adam(),
                  metrics=[LabelBinaryAccuracy()])
    model.fit(ds, epochs=20, verbose=1, callbacks=[early_stopping])
    models[lot_name] = model

We can created unified dataframe with sklearn's predictions and see where both of the models did not succeed.

In [ ]:
predict_df2 = dict()
for lot_name, lot_df in tqdm(sin_normalized_df.groupby('lot')):   
    lot_df = lot_df[['day sin', 'day cos', 'week sin', 'week cos', 'class', 'datetime']]
    ts = lot_df.pop('datetime').view('int64')
    
    arr = np.array(lot_df, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        arr,
        targets=None,
        sequence_stride=1,
        sequence_length=48, 
        shuffle=False, # Do not mix up batches
        batch_size=32
    )
    
    datetime_arr = np.array(pd.DataFrame(ts, columns=['datetime']), dtype=np.int64)
    datetime_ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        datetime_arr,
        targets=None,
        sequence_stride=1,
        sequence_length=48, 
        shuffle=False, # Do not mix up batches
        batch_size=32
    )
    
    batches = list() 
    for dt in datetime_ds:
        batches.append(dt[:, 24:, :])
        
    batch_to_datetime = tf.concat(batches, axis=0)
    
    ds = ds.map(_split_x_y)
    model = models[lot_name]
    predictions = model.predict(ds)
    predictions = tf.concat([predictions, batch_to_datetime], axis=2)
    data = tf.concat(tf.unstack(predictions, axis=0), axis=0).numpy()
    
    df = pd.DataFrame(data, columns=['day sin', 'day cos', 'week sin', 'week cos', 'prediction_dnn', 'datetime_s'])
    df['prediction_dnn'] = (df['prediction_dnn'] >= 0.5).astype(np.float32)
    df = df[['prediction_dnn', 'datetime_s']]

    df = df.groupby(by=['datetime_s']).agg({'prediction_dnn': 'mean'}).reset_index()
    df['prediction_dnn'] = (df['prediction_dnn'] >= 0.5).astype(np.float32)


    df2 = predict_df[lot_name].copy()
    df2['datetime_s'] = df2['datetime'].view('int64')

    df = df.merge(df2, on=['datetime_s'], how='inner')
    df.loc[df['prediction_dnn'] == df['class'], 'success_dnn'] = 1
    df.loc[df['prediction_dnn'] != df['class'], 'success_dnn'] = 0
    df['date'] = df['datetime'].dt.normalize()
    df['week_first_day'] = (
        ((df['date'] - FIRST_DAY_OF_FIRST_WHOLE_WEEK).dt.days / 7)
        .astype(int)
        .apply(lambda week_idx: FIRST_DAY_OF_FIRST_WHOLE_WEEK + pd.to_timedelta(timedelta(weeks=week_idx)))
    )

    df['day_hour'] = df['date'].dt.day_name() + '-' + df['datetime'].dt.hour.astype(str).str.zfill(2)
    df['any_success'] = (df['success'] + df['success_dnn'] > 0).astype(np.float32)
    df.pop('datetime_s') 
    
    
    predict_df2[lot_name] = df

predict2_with_lot_name = predict_df2.copy()
for lot in LOT_NAMES:
    predict2_with_lot_name[lot]['lot'] = lot
    predict2_with_lot_name[lot]

predict2_with_lot_name = pd.concat(predict2_with_lot_name.values())

Like before, lets run the same visualizations, this time on `any_success` so we will get better results.

In [ ]:
fig, axes = plt.subplots(len(LOT_NAMES), 1, figsize=(15, 8 * len(LOT_NAMES)), sharey=True)
fig.suptitle('Model Accuracy Per Hour', fontsize=22, y=0.895)
plt.subplots_adjust(hspace=0.4)

for i, lot in enumerate(LOT_NAMES):
    df = predict_df2[lot]
    ax = axes[i]
    plot = sns.scatterplot(
        ax=ax,
        palette=GREEN_RED_PALETTE,
        legend=False,
        x='day_hour',
        y='week_first_day',
        hue='any_success',
        s=10,
        data=df
    )
    ax.set_title(lot)

for ax in axes:
    for i, label in enumerate(ax.get_xticklabels()):
        if i % 4 == 0:
            label.set_rotation(90)
        else:
            label.set_visible(False)

Great, we can see that there is less random noise and we can definitely spot the existance of patterns. Now let's focus on week and hourly patterns:

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(13, 8))
fig.suptitle('Average Accuracy Per Hour In The Week', fontsize=22)
gs = fig.add_gridspec(3, 3)
axes = [fig.add_subplot(gs[0, :])]
for i in range(len(LOT_NAMES)):
    axes.append(fig.add_subplot(gs[1 + i // 3, i % 3]))

ax = axes[0]
sns.lineplot(
    ax=ax,
    x='day_hour',
    y='any_success',
    data=predict2_with_lot_name
)
ax.set_title('All Lots Combined')

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i + 1]
    g = sns.lineplot(
        ax=ax,
        x='day_hour',
        y='any_success',
        data=predict2_with_lot_name[predict2_with_lot_name['lot'] == lot]
    )
    ax.set_title(lot)

for ax in axes:
    ax.set_ylim(bottom=0.5, top=1)
    for i, label in enumerate(ax.get_xticklabels()):
        if i % 8 == 0:
            label.set_rotation(90)
        else:
            label.set_visible(False)

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(13, 8))
fig.suptitle('Average Accuracy Per Hour', fontsize=22)
gs = fig.add_gridspec(3, 3)
axes = [fig.add_subplot(gs[0, :])]
for i in range(len(LOT_NAMES)):
    axes.append(fig.add_subplot(gs[1 + i // 3, i % 3]))

ax = axes[0]
sns.lineplot(
    ax=ax,
    x='hour',
    y='any_success',
    data=predict2_with_lot_name
)
ax.set_title('All Lots Combined')

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i + 1]
    g = sns.lineplot(
        ax=ax,
        x='hour',
        y='any_success',
        data=predict2_with_lot_name[predict2_with_lot_name['lot'] == lot]
    )
    ax.set_title(lot)

for ax in axes:
    ax.set_ylim(bottom=0.5, top=1)
    for i, label in enumerate(ax.get_xticklabels()):
        label.set_rotation(90)

Here we go! this time we can see that Basel and Asuta were harder to predict specifically very early at the morning, when someone leaves its parking spot and there is barely traffic, and around 6-7pm, when people come back from work and the parking lots are queuing up.

In [ ]:
weekly_predict2_df = (
    predict2_with_lot_name
    .groupby(by=['week_first_day', 'lot'])
    .agg({'any_success': 'mean'})
    .reset_index()
)

fig = plt.figure(constrained_layout=True, figsize=(13, 8))
fig.suptitle('Average Accuracy Per Week Over Time', fontsize=22)
gs = fig.add_gridspec(3, 3)
axes = [fig.add_subplot(gs[0, :])]
for i in range(len(LOT_NAMES)):
    axes.append(fig.add_subplot(gs[1 + i // 3, i % 3]))

ax = axes[0]
sns.lineplot(
    ax=ax,
    x='week_first_day',
    marker='o',
    y='any_success',
    data=weekly_predict2_df
)
ax.set_title('All Lots Combined')

for i, lot in enumerate(LOT_NAMES):
    ax = axes[i + 1]
    sns.lineplot(
        ax=ax,
        x='week_first_day',
        marker='o',
        y='any_success',
        data=weekly_predict2_df[weekly_predict2_df['lot'] == lot]
    )
    ax.set_title(lot)

for ax in axes:
    ax.set_ylim(bottom=0.7, top=1)
    for i, label in enumerate(ax.get_xticklabels()):
        label.set_rotation(90)

Over time, we can spot some anomalies usually around holidays. More over, we can see some spikes on uncertainty around April 2020, September 2020 and December 2020 - months with heavy covid restrictions!

# Final Words
We had great time working on this intersting project! Each and every aspect of it has taught us new skills. We practiced data visualizations, worked with classifiers and tweaked some deep learning models. We also learned techniques to cross-validate our results to reduce bias.<br>
Even though we decided to end this project here, we believe there are many ways we can continue exploring from here. We might be able to find correlations with other data such as NLP analysis of news. There is also much room for improvement in our models - we only tested few configurations with not-so-effective computational efficiency. We could push our anomaly detection even further and create 2nd degree models that predicts anomalies using our trained models!<br>
Since we're staying in Tel-Aviv, we will keep collecting data and continue this project and on our free time. We already have almost two years of data, it would be worth checking it again once we have even more data.